In [46]:
import torch
import torch.nn.functional as F

import matplotlib.pyplot as plt
%matplotlib inline

In [47]:
# read all the words
words = open('names.txt', 'r').read().splitlines()

In [48]:
len(words)

32033

In [49]:
# build the vocabulary of chars and mapping to and from integers
chars = sorted(list(set(''.join(words))))

stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0

itos = {i:s for s,i in stoi.items()}

In [50]:
# buid the data set

BLOCK_SIZE = 3 # context length: how many char we take to predict next char

X, Y = [], []

for w in words:
#     print(w)
    context = [0] * BLOCK_SIZE
    
    for ch in list(w) + ['.']:
        ix = stoi[ch]
        
        X.append(context)
        Y.append(ix)
        
#         print(''.join(itos[i] for i in context), '--->', itos[ix])
        
        # update the context with next char
        context = context[1:] + [ix]
        
X = torch.tensor(X)
Y = torch.tensor(Y)

In [51]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([228146, 3]), torch.int64, torch.Size([228146]), torch.int64)

In [59]:
# generator
g = torch.Generator().manual_seed(2147483647)

In [66]:
# create embeddings
C = torch.randn((27, 2), generator=g)

# hidden layers
W1 = torch.randn((6, 100), generator=g)
b1 = torch.randn(100, generator=g)

W2 = torch.randn((100, 27), generator=g)
b2 = torch.randn(27, generator=g)

parameters = [C, W1, b1, W2, b2]

In [67]:
# number of parameters

sum(p.nelement() for p in parameters)

3481

In [68]:
for p in parameters:
    p.requires_grad = True

In [69]:
lre = torch.linspace(-3, 0, 1000)
lrs = 10**lre

In [84]:
lri = []
lossi = []

for i in range(10000):
    
    # minibatch construct
    ix = torch.randint(0, X.shape[0], (32, ))
    
    # forward pass

    # embedding
    emb = C[X[ix]]

    # tanh
    h = torch.tanh(emb.view(-1, 6) @ W1 + b1)

    # logits
    logits = h @ W2 + b2

    # # soft max
    # counts = logits.exp()
    # probs = counts / counts.sum(1, keepdims=True)
    # # loss
    # loss = -probs[torch.arange(32), Y].log().mean()

    loss = F.cross_entropy(logits, Y[ix])
    
    # backward pass
    for p in parameters:
        p.grad = None

    loss.backward()

    # update 
    lr = 0.01
    # optimim
    for p in parameters:
        p.data += -lr * p.grad

    # track stats
#     lri.append(10**0.1)
#     lossi.append(loss.item())
    
print(loss.item())

2.4826295375823975


In [85]:
# plt.plot(lri, lossi)

In [86]:
emb = C[X]
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Y)
loss

tensor(2.2954, grad_fn=<NllLossBackward0>)